<a href="https://www.kaggle.com/code/vtrackstar/most-statistically-similar-nba-seasons?scriptVersionId=200922090" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nba-aba-baa-stats/Advanced.csv
/kaggle/input/nba-aba-baa-stats/Player Season Info.csv
/kaggle/input/nba-aba-baa-stats/Player Career Info.csv
/kaggle/input/nba-aba-baa-stats/Player Shooting.csv
/kaggle/input/nba-aba-baa-stats/Opponent Stats Per Game.csv
/kaggle/input/nba-aba-baa-stats/Player Totals.csv
/kaggle/input/nba-aba-baa-stats/Opponent Stats Per 100 Poss.csv
/kaggle/input/nba-aba-baa-stats/Per 100 Poss.csv
/kaggle/input/nba-aba-baa-stats/Team Summaries.csv
/kaggle/input/nba-aba-baa-stats/Player Per Game.csv
/kaggle/input/nba-aba-baa-stats/All-Star Selections.csv
/kaggle/input/nba-aba-baa-stats/End of Season Teams (Voting).csv
/kaggle/input/nba-aba-baa-stats/Player Play By Play.csv
/kaggle/input/nba-aba-baa-stats/End of Season Teams.csv
/kaggle/input/nba-aba-baa-stats/Team Stats Per 100 Poss.csv
/kaggle/input/nba-aba-baa-stats/Per 36 Minutes.csv
/kaggle/input/nba-aba-baa-stats/Opponent Totals.csv
/kaggle/input/nba-aba-baa-stats/Team Abbrev.csv
/kaggle/input/nba-aba-b

In [2]:
playerpg = pd.read_csv('/kaggle/input/nba-aba-baa-stats/Player Per Game.csv')
playerpbp = pd.read_csv('/kaggle/input/nba-aba-baa-stats/Player Play By Play.csv')
playeraw = pd.read_csv('/kaggle/input/nba-aba-baa-stats/Player Award Shares.csv')
playersh = pd.read_csv('/kaggle/input/nba-aba-baa-stats/Player Shooting.csv')

print(playerpg.columns)
print(playerpbp.columns)
print(playeraw.columns)
print(playersh.columns)

Index(['seas_id', 'season', 'player_id', 'player', 'birth_year', 'pos', 'age',
       'experience', 'lg', 'tm', 'g', 'gs', 'mp_per_game', 'fg_per_game',
       'fga_per_game', 'fg_percent', 'x3p_per_game', 'x3pa_per_game',
       'x3p_percent', 'x2p_per_game', 'x2pa_per_game', 'x2p_percent',
       'e_fg_percent', 'ft_per_game', 'fta_per_game', 'ft_percent',
       'orb_per_game', 'drb_per_game', 'trb_per_game', 'ast_per_game',
       'stl_per_game', 'blk_per_game', 'tov_per_game', 'pf_per_game',
       'pts_per_game'],
      dtype='object')
Index(['seas_id', 'season', 'player_id', 'player', 'birth_year', 'pos', 'age',
       'experience', 'lg', 'tm', 'g', 'mp', 'pg_percent', 'sg_percent',
       'sf_percent', 'pf_percent', 'c_percent',
       'on_court_plus_minus_per_100_poss', 'net_plus_minus_per_100_poss',
       'bad_pass_turnover', 'lost_ball_turnover', 'shooting_foul_committed',
       'offensive_foul_committed', 'shooting_foul_drawn',
       'offensive_foul_drawn', 'points_gener

In [3]:
con_perg = playerpg[['seas_id', 'season', 'player', 'tm', 'experience', 'age', 'pos','g', 'mp_per_game', 'fg_per_game',
       'fga_per_game', 'fg_percent', 'x3p_per_game', 'x3pa_per_game',
       'x3p_percent', 'x2p_per_game', 'x2pa_per_game', 'x2p_percent',
       'e_fg_percent', 'ft_per_game', 'fta_per_game', 'ft_percent',
       'orb_per_game', 'drb_per_game', 'trb_per_game', 'ast_per_game',
       'stl_per_game', 'blk_per_game', 'tov_per_game', 'pf_per_game',
       'pts_per_game']].copy()
con_pbp = playerpbp[['seas_id', 'pg_percent', 'sg_percent',
       'sf_percent', 'pf_percent', 'c_percent',
       'on_court_plus_minus_per_100_poss', 'net_plus_minus_per_100_poss',
       'bad_pass_turnover', 'lost_ball_turnover', 'shooting_foul_committed',
       'offensive_foul_committed', 'shooting_foul_drawn',
       'offensive_foul_drawn', 'points_generated_by_assists', 'and1',
       'fga_blocked']].copy()
con_award = playeraw[['seas_id', 'first', 'pts_won', 'pts_max',
       'share', 'winner']].copy()
con_shot = playersh[['seas_id', 'avg_dist_fga',
       'percent_fga_from_x2p_range', 'percent_fga_from_x0_3_range',
       'percent_fga_from_x3_10_range', 'percent_fga_from_x10_16_range',
       'percent_fga_from_x16_3p_range', 'percent_fga_from_x3p_range',
       'fg_percent_from_x2p_range', 'fg_percent_from_x0_3_range',
       'fg_percent_from_x3_10_range', 'fg_percent_from_x10_16_range',
       'fg_percent_from_x16_3p_range', 'fg_percent_from_x3p_range',
       'percent_assisted_x2p_fg', 'percent_assisted_x3p_fg',
       'percent_dunks_of_fga', 'num_of_dunks', 'percent_corner_3s_of_3pa',
       'corner_3_point_percent', 'num_heaves_attempted', 'num_heaves_made']].copy()

merged_player = con_perg.merge(con_pbp, on='seas_id', how='outer') \
               .merge(con_shot, on='seas_id', how='outer')
print(merged_player.head())
print(merged_player.columns)

   seas_id  season         player   tm  experience   age  pos   g  \
0        1    1947   Al Brightman  BOS           1  23.0    F  58   
1        2    1947      Al Lujack  WSC           1  26.0    F   5   
2        3    1947    Al Negratti  WSC           1  25.0  F-C  11   
3        4    1947    Angelo Musi  PHW           1  28.0    G  60   
4        5    1947  Ariel Maughan  DTF           1  23.0    F  59   

   mp_per_game  fg_per_game  ...  fg_percent_from_x16_3p_range  \
0          NaN          3.8  ...                           NaN   
1          NaN          0.2  ...                           NaN   
2          NaN          1.2  ...                           NaN   
3          NaN          3.8  ...                           NaN   
4          NaN          3.8  ...                           NaN   

   fg_percent_from_x3p_range  percent_assisted_x2p_fg  \
0                        NaN                      NaN   
1                        NaN                      NaN   
2                

In [4]:
preshai = merged_player[merged_player['player'] == 'Shai Gilgeous-Alexander']  # For string value
preshai.head()

,seas_id,season,player,tm,experience,age,pos,g,mp_per_game,fg_per_game,...,fg_percent_from_x16_3p_range,fg_percent_from_x3p_range,percent_assisted_x2p_fg,percent_assisted_x3p_fg,percent_dunks_of_fga,num_of_dunks,percent_corner_3s_of_3pa,corner_3_point_percent,num_heaves_attempted,num_heaves_made
28162,28163,2019,Shai Gilgeous-Alexander,LAC,1,20.0,PG,82,26.5,4.2,...,0.436,0.367,0.310,0.902,0.028,19.0,0.460,0.406,1.0,0.0
28838,28839,2020,Shai Gilgeous-Alexander,OKC,2,21.0,SG,70,34.7,6.8,...,0.507,0.347,0.235,0.540,0.019,19.0,0.127,0.438,0.0,0.0
29540,29541,2021,Shai Gilgeous-Alexander,OKC,3,22.0,SG,35,33.7,8.2,...,0.360,0.418,0.074,0.296,0.018,10.0,0.076,0.462,0.0,0.0
30336,30337,2022,Shai Gilgeous-Alexander,OKC,4,23.0,PG,56,34.7,8.5,...,0.490,0.300,0.137,0.303,0.023,23.0,0.077,0.391,0.0,0.0
31045,31046,2023,Shai Gilgeous-Alexander,OKC,5,24.0,PG,68,35.5,10.4,...,0.396,0.345,0.201,0.310,0.039,50.0,0.101,0.235,1.0,0.0


In [5]:
# Count the number of NaN values in each column
nan_counts = merged_player.isna().sum()

# Display columns with NaN values
nan_columns = nan_counts[nan_counts > 0]

# Print the columns and their respective NaN counts
print(nan_columns)

age                                    22
mp_per_game                          1083
fg_percent                            157
x3p_per_game                         6352
x3pa_per_game                        6352
x3p_percent                         10528
x2p_percent                           239
e_fg_percent                          157
ft_percent                           1291
orb_per_game                         4657
drb_per_game                         4657
trb_per_game                          894
stl_per_game                         5626
blk_per_game                         5625
tov_per_game                         5635
pg_percent                          26043
sg_percent                          23992
sf_percent                          23605
pf_percent                          23143
c_percent                           24970
on_court_plus_minus_per_100_poss    15094
net_plus_minus_per_100_poss         15094
bad_pass_turnover                   15085
lost_ball_turnover                

In [6]:
similarity_columns = [
    'fg_per_game', 'fga_per_game', 'fg_percent',
       'x3p_per_game', 'x3pa_per_game', 'x3p_percent', 'x2p_per_game',
       'x2pa_per_game', 'x2p_percent', 'e_fg_percent', 'ft_per_game',
       'fta_per_game', 'ft_percent', 'orb_per_game', 'drb_per_game',
       'trb_per_game', 'ast_per_game', 'stl_per_game', 'blk_per_game',
       'tov_per_game', 'pf_per_game', 'pts_per_game',       
       'bad_pass_turnover', 'lost_ball_turnover', 'shooting_foul_committed',
       'offensive_foul_committed', 'shooting_foul_drawn',
       'offensive_foul_drawn', 'points_generated_by_assists', 'and1',
       'fga_blocked', 'avg_dist_fga', 'percent_assisted_x2p_fg',
       'percent_assisted_x3p_fg', 'percent_dunks_of_fga', 'num_of_dunks',
       'percent_corner_3s_of_3pa', 'corner_3_point_percent'
]

In [7]:
shai = merged_player[merged_player['player'] == 'Shai Gilgeous-Alexander']
pd.set_option('display.max_columns', None)  # Show all columns

print(shai)

       seas_id  season                   player   tm  experience   age pos  \
28162    28163    2019  Shai Gilgeous-Alexander  LAC           1  20.0  PG   
28838    28839    2020  Shai Gilgeous-Alexander  OKC           2  21.0  SG   
29540    29541    2021  Shai Gilgeous-Alexander  OKC           3  22.0  SG   
30336    30337    2022  Shai Gilgeous-Alexander  OKC           4  23.0  PG   
31045    31046    2023  Shai Gilgeous-Alexander  OKC           5  24.0  PG   
31768    31769    2024  Shai Gilgeous-Alexander  OKC           6  25.0  PG   

        g  mp_per_game  fg_per_game  fga_per_game  fg_percent  x3p_per_game  \
28162  82         26.5          4.2           8.7       0.476           0.6   
28838  70         34.7          6.8          14.5       0.471           1.2   
29540  35         33.7          8.2          16.1       0.508           2.0   
30336  56         34.7          8.5          18.8       0.453           1.6   
31045  68         35.5         10.4          20.3       0.

In [8]:
pre = merged_player.fillna(0).copy()
player_stats = pre[similarity_columns].dropna(how='all')

# Alternatively, keep rows with any non-NaN values
# player_stats = merged_player[similarity_columns].dropna(how='any')

In [9]:
# Normalize the data
scaler = StandardScaler()
player_stats_scaled = scaler.fit_transform(player_stats)

# Choose a specific row (e.g., the player to compare)
chosen_row_index = 31768  # Adjust as needed
chosen_row = player_stats_scaled[chosen_row_index].reshape(1, -1)

# Calculate pairwise distances from the chosen row to all others
distances = pairwise_distances(player_stats_scaled, chosen_row, metric='euclidean').flatten()

In [10]:
short = ['season', 'player', 'experience', 'age', 'tm', 'pos', 'g', 'ft_percent', 'orb_per_game', 'drb_per_game',
       'trb_per_game', 'ast_per_game', 'stl_per_game', 'blk_per_game',
       'tov_per_game', 'pts_per_game', 'similarity_score']
shai_comp = merged_player.copy()
shai_comp['similarity_score'] = distances
shai_sorted = shai_comp.sort_values(by='similarity_score')
best_shai = shai_sorted.drop_duplicates(subset='player', keep='first')
best_shai_f = best_shai[short]
best_shai_f.head(6)

,season,player,experience,age,tm,pos,g,ft_percent,orb_per_game,drb_per_game,trb_per_game,ast_per_game,stl_per_game,blk_per_game,tov_per_game,pts_per_game,similarity_score
31768,2024,Shai Gilgeous-Alexander,6,25.0,OKC,PG,75,0.874,0.9,4.7,5.5,6.2,2.0,0.9,2.2,30.1,0.000000
29829,2022,DeMar DeRozan,13,32.0,CHI,PF,76,0.877,0.7,4.4,5.2,4.9,0.9,0.3,2.4,27.9,4.245825
21891,2009,Kobe Bryant,13,30.0,LAL,SG,82,0.856,1.1,4.1,5.2,4.9,1.5,0.5,2.6,26.8,4.355879
26586,2017,Jimmy Butler,6,27.0,CHI,SF,76,0.865,1.7,4.5,6.2,5.5,1.9,0.4,2.1,23.9,4.584857
18698,2003,Tracy McGrady,6,23.0,ORL,SG,75,0.793,1.6,4.9,6.5,5.5,1.7,0.8,2.6,32.1,4.990954
29003,2021,Bradley Beal,9,27.0,WAS,SG,60,0.889,1.2,3.5,4.7,4.4,1.2,0.4,3.1,31.3,5.145955
